In [1]:
import random

import pandas as pd, numpy as np, string, re
import matplotlib.pyplot as plt
from matplotlib import dates as mpl_dates
import matplotlib.font_manager as fm 
from datetime import timedelta, datetime as dt
import matplotlib.gridspec as gridspec
%matplotlib inline
# %matplotlib notebook

pd.set_option('display.max_columns', 30)

In [2]:
def get_colors(cmap, n, start=0.0, stop=1.0, alpha=1.0, reverse=False):
    import matplotlib.cm as cm, numpy as np 
    colors = [cm.get_cmap(cmap)(x) for x in np.linspace(start, stop, n)]
    colors = [(r, g, b, alpha) for r, g, b, _ in colors]
    return list(reversed(colors)) if reverse else colors

In [15]:
plt.style.use('fivethirtyeight')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
print('\n'.join(color for color in colors)) 

#008fd5
#fc4f30
#e5ae38
#6d904f
#8b8b8b
#810f7c


In [3]:
family = 'DejaVu Sans'
title_font = fm.FontProperties(family=family, style='normal', size=20, weight='normal', stretch='normal')
label_font = fm.FontProperties(family=family, style='normal', size=16, weight='normal', stretch='normal')
ticks_font = fm.FontProperties(family=family, style='normal', size=12, weight='normal', stretch='normal')
ticks_font_h = fm.FontProperties(family=family, style='normal', size=10.5, weight='normal', stretch='normal')

In [4]:
features = pd.read_csv('D:\development\music-analysis\spotify\Spreadsheets\Scrobbles with Features (69126).csv')

In [5]:
features['timestamp'] = pd.to_datetime(features['datetime'])
features = features.drop('datetime', axis=1)
features['timestamp'] = features['timestamp'].dt.tz_localize('UTC')
features['timestamp'] = features['timestamp'].dt.tz_convert('Asia/Kolkata')
features['timestamp'] = features['timestamp'].dt.tz_localize(None)

In [6]:
features['year'] = features['timestamp'].dt.year
features['month'] = features['timestamp'].dt.month
features['year-month'] = features['timestamp'].apply(lambda time: '{}-{:02}'.format(time.year, time.month))
features['week'] = features['timestamp'].dt.week
features['day'] =  features['timestamp'].dt.day
features['hour'] = features['timestamp'].dt.hour
features['dow'] = features['timestamp'].dt.dayofweek


dmap = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}

# features['dow'] = features['dow'].map(dmap)

<ipython-input-6-453e66adfce6>:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  features['week'] = features['timestamp'].dt.week


In [7]:
keymap = {0: 'C', 1: 'C♯/D♭', 2: 'D', 3: 'D♯/E♭', 4: 'E/F♭', 5: 'F/E♯', 6: 'F♯/G♭', 7: 'G', 8: 'G♯/A♭', 9: 'A', 10: 'A♯/B♭',  11: 'B/C♭'}

features['key'] = features['key'].map(keymap)

features['length in mins'] = features['length']/60000

In [8]:
features = features.set_index('timestamp')

In [9]:
features.sample(5)

,artist,track,album,release_date,genres,length,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year,month,year-month,week,day,hour,dow,length in mins
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-03-28 02:13:59,John Mayer Trio,Who Did You Think I Was - Live In Concert,TRY!,2005-09-19,['modern blues'],189160.0,42.0,0.232,0.9680,G,-5.597,1.0,0.1020,0.000379,0.136,0.938,0.419,188.686,4.0,2019,3,2019-03,13,28,2,3,3.152667
2019-09-24 09:39:12,The Kooks,Naïve,Acoustic Rock Covers,2016-07-10,[],209204.0,18.0,0.621,0.3330,G,-8.659,1.0,0.0418,0.681000,0.000,0.115,0.631,105.869,4.0,2019,9,2019-09,39,24,9,1,3.486733
2017-02-24 07:01:49,Mojave 3,Got My Sunshine,Excuses for Travellers,2000-05-15,"['dream pop', 'nu gaze', 'slowcore']",259933.0,33.0,0.303,0.2910,C,-10.505,1.0,0.0304,0.556000,0.489,0.325,0.206,126.309,4.0,2017,2,2017-02,8,24,7,4,4.332217
2016-12-29 06:45:10,Tycho,Phoenix,Past Is Prologue,2006-10-25,"['chillwave', 'downtempo', 'indietronica', 'in...",86066.0,29.0,0.270,0.0703,D,-25.854,0.0,0.0399,0.971000,0.705,0.100,0.180,72.764,3.0,2016,12,2016-12,52,29,6,3,1.434433
2017-06-21 17:01:23,Porcupine Tree,Lazarus,Deadwing,2005-03-28,"['alternative metal', 'art rock', 'progressive...",258586.0,53.0,0.354,0.5810,A,-6.074,1.0,0.0302,0.203000,0.655,0.177,0.337,171.961,4.0,2017,6,2017-06,25,21,17,2,4.309767


In [174]:
seventeen = features.loc['2017']
eighteen = features.loc['2018']
nineteen = features.loc['2019']

In [175]:
monthly_17 = seventeen.resample('M').mean()
monthly_18 = eighteen.resample('M').mean()
monthly_19 = nineteen.resample('M').mean()

In [176]:
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

In [177]:
mean_valence = features['valence'].mean()
mean_energy = features['energy'].mean()
mean_danceability = features['danceability'].mean()
mean_length = features['length in mins'].mean()

In [178]:
valence_diff = monthly_17['valence'].values - mean_valence
energy_diff = monthly_17['energy'].values - mean_energy
danceability_diff = monthly_17['danceability'].values - mean_danceability

valence_diff_18 = monthly_18['valence'].values - mean_valence
energy_diff_18 = monthly_18['energy'].values - mean_energy
danceability_diff_18 = monthly_18['danceability'].values - mean_danceability


valence_diff_19 = monthly_19['valence'].values - mean_valence
energy_diff_19 = monthly_19['energy'].values - mean_energy
danceability_diff_19 = monthly_19['danceability'].values - mean_danceability


In [179]:
monthly_17 = monthly_17.drop(['length', 'time_signature', 'year', 'month', 'day', 'week', 'hour', 'dow', 'length in mins', 'tempo'], axis=1)
monthly_18 = monthly_18.drop(['length', 'time_signature', 'year', 'month', 'day', 'week', 'hour', 'dow', 'length in mins', 'tempo'], axis=1)
monthly_19 = monthly_19.drop(['length', 'time_signature', 'year', 'month', 'day', 'week', 'hour', 'dow', 'length in mins', 'tempo'], axis=1)

In [180]:
monthly_17['valence_diff'] = monthly_17['valence'].values - mean_valence
monthly_17['energy_diff'] = monthly_17['energy'].values - mean_energy
monthly_17['danceability_diff'] = monthly_17['danceability'].values - mean_danceability

monthly_18['valence_diff'] = monthly_18['valence'].values - mean_valence
monthly_18['energy_diff'] = monthly_18['energy'].values - mean_energy
monthly_18['danceability_diff'] = monthly_18['danceability'].values - mean_danceability


monthly_19['valence_diff'] = monthly_19['valence'].values - mean_valence
monthly_19['energy_diff'] = monthly_19['energy'].values - mean_energy
monthly_19['danceability_diff'] = monthly_19['danceability'].values - mean_danceability

In [181]:
monthly_17['depressed'] = [1 if x < 0 else 0 for x in monthly_17['energy_diff']]
monthly_18['depressed'] = [1 if x < 0 else 0 for x in monthly_18['energy_diff']]
monthly_19['depressed'] = [1 if x < 0 else 0 for x in monthly_19['energy_diff']]

In [182]:
labeled_monthly_17 = monthly_17[['popularity', 'danceability', 'energy', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'valence_diff', 'energy_diff',
       'danceability_diff', 'depressed']]

labeled_monthly_18 = monthly_18[['popularity', 'danceability', 'energy', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'valence_diff', 'energy_diff',
       'danceability_diff', 'depressed']]


labeled_monthly_19 = monthly_19[['popularity', 'danceability', 'energy', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'valence_diff', 'energy_diff',
       'danceability_diff', 'depressed']]

In [11]:
try:    
    labeled_monthly_17 = pd.read_csv('monthly_17_labeled.csv')
    labeled_monthly_18 = pd.read_csv('monthly_18_labeled.csv')
    labeled_monthly_19 = pd.read_csv('monthly_19_labeled.csv')
except:
    labeled_monthly_17.to_csv('monthly_17_labeled.csv', index=False)
    labeled_monthly_18.to_csv('monthly_18_labeled.csv', index=False)
    labeled_monthly_19.to_csv('monthly_19_labeled.csv', index=False)


In [12]:
try:
    merged = pd.read_csv('merged_monthly.csv')
except Exception:
    merged = pd.concat([labeled_monthly_17, labeled_monthly_18, labeled_monthly_19])
    merged.to_csv('merged_monthly.csv', index=False)
merged

,popularity,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,valence_diff,energy_diff,danceability_diff,depressed
0,46.125255,0.519424,0.541410,-9.836375,0.675235,0.049379,0.381088,0.287946,0.164311,0.323573,-0.019213,0.019135,0.016533,0
1,45.716087,0.511970,0.522143,-10.052432,0.688855,0.047033,0.389196,0.238978,0.160890,0.334514,-0.008273,-0.000131,0.009079,1
2,50.534207,0.471335,0.525519,-9.921551,0.662300,0.042511,0.399270,0.259213,0.160628,0.330857,-0.011929,0.003244,-0.031556,0
3,53.047447,0.480878,0.525809,-9.690519,0.675513,0.044225,0.393007,0.253387,0.170069,0.340054,-0.002732,0.003534,-0.022013,0
4,48.990761,0.477147,0.455902,-10.403747,0.723273,0.040918,0.477585,0.205936,0.168806,0.325908,-0.016878,-0.066373,-0.025744,1
5,47.380355,0.476665,0.466758,-10.829071,0.636263,0.042093,0.401824,0.270899,0.160739,0.308759,-0.034027,-0.055517,-0.026226,1
6,43.418487,0.488976,0.486048,-10.770816,0.634165,0.043802,0.380066,0.285057,0.177663,0.293967,-0.048819,-0.036227,-0.013915,1
7,44.540065,0.480753,0.482756,-10.709914,0.651455,0.045958,0.414039,0.288865,0.169247,0.299165,-0.043621,-0.039518,-0.022137,1
8,43.370234,0.490265,0.524012,-10.286567,0.619660,0.045527,0.366856,0.240311,0.240903,0.326403,-0.016384,0.001737,-0.012626,0
9,43.923232,0.490996,0.513355,-10.589053,0.647980,0.046558,0.384422,0.299652,0.218995,0.315730,-0.027056,-0.008920,-0.011895,1


<h1>TRAINING A MODEL</h1>

In [14]:
X = merged.iloc[:, 1:-1].values
y = merged.iloc[:, -1].values

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33)

In [16]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [17]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 3, metric = 'minkowski', p = 2)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [18]:
predictions = knn.predict(X_test)
print(np.concatenate((predictions.reshape(len(predictions),1), y_test.reshape(len(y_test),1)),1))

[[1 1]
 [0 0]
 [1 1]
 [0 0]
 [0 1]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [0 1]
 [1 0]
 [1 1]]


In [19]:
from sklearn.metrics import confusion_matrix, accuracy_score
print(confusion_matrix(y_test, predictions))
accuracy_score(y_test, predictions)

[[5 1]
 [2 4]]


0.75